# INA-190-Q1 ranges calculation

In [42]:
# Constants
max_r = 1000
adc_resolution = 12
adc_swing = 3.3
adc_step = adc_swing / 2**adc_resolution

ina_gain = 50

In [43]:
def to_si(nr, unit):
    """
    Convert a number to a SI representation
    to_si(0.001, "A") => "1 mA"
    """
    tmp = nr
    decades = 0
    if nr < 1.0:
        while tmp < 1.0:
            decades += 1
            tmp *= 1000
        prefixes = ["", "m", "μ", "n"]
        assert decades < len(prefixes)
        return f"{tmp:.3f} {prefixes[decades]}{unit}"
    else:
        while tmp >= 1000.0:
            decades += 1
            tmp /= 1000.0
        prefixes = ["", "k", "M", "G"]
        assert decades < len(prefixes)
        return f"{tmp:.3f} {prefixes[decades]}{unit}"

In [44]:
def calc_r_sense(i_max, max_drop):
    """
    i_min: min current to be detected (A)
    i_max: max current to be detected (A)
    max_drop: max voltage drop accross the resistor (V)
    """
    
    # From datasheet:
    # r_sense < v_sp / (i_max * gain)
    # v_out_min = i_min * r_sense * gain
    v_sp = adc_swing - (40 / 1000) # pick worst case from datasheet
    r_sense = min(min(v_sp / (i_max * ina_gain), max_r), max_drop / i_max)
    
    return r_sense

In [45]:
def error_analysis(current, r_sense):
    # Code from the website of TI
    import math

    # user chosen
    temp = 25
    cmrr_user_voltage = 3.3 
    supply_voltage = 3.3
    psrr = 1
    #current = 0.01
    #r_sense = 100e-3

    # see datasheet 
    DEFAULT_TEMP = 25.0
    CMRR = 132
    gain_error = 0.002
    v_os_max = 15e-6
    v_drift = 0.01e-6
    v_os_temp_err = v_os_max + v_drift * abs(temp - DEFAULT_TEMP)

    cmrr_specified = 12.0
    voltage_specified = 1.8
    CMRR_temp_err = abs( 1 / 10**(CMRR / 20) * (cmrr_user_voltage - cmrr_specified))
    psrr_err = psrr / 1e6 * abs(supply_voltage - voltage_specified)
    v_shunt = current * r_sense

    total_error = math.sqrt( ((v_os_max + CMRR_temp_err + psrr_err)/(v_shunt))**2 +
                              gain_error**2)

    return total_error * 100

In [46]:
def info_for_range(i_min, i_max, r_sense, r_tolerance):
    """
    Print some stats about the returned params
    """
    v_out_min = i_min * r_sense * ina_gain
    v_out_max = i_max * r_sense * ina_gain
    v_drop_min = i_min * r_sense
    v_drop_max = i_max * r_sense
    steps_v_min = v_out_min / adc_step
    error_min = error_analysis(i_min, r_sense)
    error_max = error_analysis(i_max, r_sense)
    
    def calc_tolerance_error(i, r_sense, tolerance):
        valb = i * (r_sense) * ina_gain
        val1 = i * (r_sense + r_sense * r_tolerance) * ina_gain
        val2 = i * (r_sense * (1.0 - r_tolerance)) * ina_gain
        print("With tolerance {}: {}, {} ({}, {})".format(tolerance,
                                                          valb / val1,
                                                          valb / val2,
                                                          val1, val2))
            
    print("Range: [{}, {}], resistor: {}".format(
                to_si(i_min, "A"),
                to_si(i_max, "A"),
                to_si(r_sense, "Ohm")))
    print("Vdrop swing: [{}, {}]".format(
                to_si(v_drop_min, "V"),
                to_si(v_drop_max, "V")))
    print("Vout swing: [{}, {}]".format(
                to_si(v_out_min, "V"),
                to_si(v_out_max, "V")))
    print("Steps for v_out_min: {}".format(steps_v_min))
    print("Error range: [{:},{}]%".format(
                error_min,
                error_max))
    calc_tolerance_error(i_min, r_sense, r_tolerance)
    calc_tolerance_error(i_max, r_sense, r_tolerance)
    

In [47]:
def calc_range(i_min, i_max, tolerance=0.05):
    r_sense = calc_r_sense(i_max, 50e-3)
    info_for_range(i_min, i_max, r_sense, tolerance)

In [48]:
calc_range(1e-6, 100e-6)

Range: [1.000 μA, 100.000 μA], resistor: 500.000 Ohm
Vdrop swing: [500.000 μV, 50.000 mV]
Vout swing: [25.000 mV, 2.500 V]
Steps for v_out_min: 31.030303030303035
Error range: [3.7424162012743087,0.20346146539911691]%
With tolerance 0.05: 0.9523809523809524, 1.0526315789473684 (0.02625, 0.02375)
With tolerance 0.05: 0.9523809523809522, 1.0526315789473684 (2.6250000000000004, 2.375)


In [49]:
calc_range(100e-6, 10e-3)

Range: [100.000 μA, 10.000 mA], resistor: 5.000 Ohm
Vdrop swing: [500.000 μV, 50.000 mV]
Vout swing: [25.000 mV, 2.500 V]
Steps for v_out_min: 31.030303030303035
Error range: [3.7424162012743087,0.20346146539911691]%
With tolerance 0.05: 0.9523809523809523, 1.0526315789473684 (0.026250000000000002, 0.02375)
With tolerance 0.05: 0.9523809523809523, 1.0526315789473684 (2.625, 2.375)


In [55]:
calc_range(10e-3, 500e-3)

Range: [10.000 mA, 500.000 mA], resistor: 100.000 mOhm
Vdrop swing: [1.000 mV, 50.000 mV]
Vout swing: [50.000 mV, 2.500 V]
Steps for v_out_min: 62.06060606060607
Error range: [1.8792072147291548,0.20346146539911691]%
With tolerance 0.05: 0.9523809523809523, 1.0526315789473684 (0.052500000000000005, 0.0475)
With tolerance 0.05: 0.9523809523809522, 1.0526315789473684 (2.6250000000000004, 2.375)


In [10]:
# ===========================

In [11]:
info_for_range(100e-6, 10e-3, 20)

Range: [100.000 μA, 10.000 mA], resistor: 20.000 Ohm
Vdrop swing: [2.000 mV, 200.000 mV]
Vout swing: [50.000 mV, 5.000 V]
Steps for v_out_min: 40.96
Error range: [1.0387036820641102,0.20025955790901304]%


In [12]:
info_for_range(1e-3, 300e-3, 660e-3)

Range: [1.000 mA, 300.000 mA], resistor: 660.000 mOhm
Vdrop swing: [660.000 μV, 198.000 mV]
Vout swing: [16.500 mV, 4.950 V]
Steps for v_out_min: 13.5168
Error range: [3.0951565277764983,0.20026482450138747]%
